In [2]:
import pandas as pd
import numpy as np

import json

In [5]:
_json = json.load(open("./eu_emission_mother_load.json"))
eu_brand_model = pd.json_normalize(_json['results'])
eu_brand_model.to_csv("./eu_emission_mother_load.csv", index=False)

In [3]:
eu_brand_model.sort_values('Emission')

,Make,Fuel,Typ,Model,Cnt,Emission,year
8384,PEUGEOT,ELECTRIC,U,2008 GT EV,992,0.0,2020
10958,NISSAN,ELECTRIC,ZE1,LEAF E + N-TEC,648,0.0,2020
2700,NISSAN,electric,ZE1,LEAF TEKNA,4666,0.0,2019
16344,HYUNDAI,electric,OS,KONA PREMIUM EV AUTO,316,0.0,2019
6936,VOLKSWAGEN,ELECTRIC,E1,ID3 LIFE,1318,0.0,2020
...,...,...,...,...,...,...,...
19993,MITSUBISHI J,petrol,GK0,MITSUBISHI ECLIPSE CROSS,212,NaN,2019
19994,,DIESEL,J-A,MOKKAX,212,NaN,2017
19997,BMW,DIESEL,G3X,X3 XDRIVE30D,212,NaN,2017
19998,MERCEDES BENZ,Diesel,218,CLASSE CLS,212,NaN,2012


## Data Cleaning

In [ ]:
# json.dump({make:make  for make in eu_brand_model['Make'].unique()}, open("make_renaming.json", "w"), indent=4) 

In [ ]:
# json.dump({f:f  for f in eu_brand_model['Fuel'].unique()}, open("fuel_renaming.json", "w"), indent=4) 

In [5]:
eu_brand_model['Fuel'] = eu_brand_model['Fuel'].str.upper()
eu_brand_model['Fuel'] = eu_brand_model['Fuel'].str.strip()
eu_brand_model['Make'] = eu_brand_model['Make'].str.upper()
eu_brand_model['Make'] = eu_brand_model['Make'].str.strip()
eu_brand_model['Model'] = eu_brand_model['Model'].str.upper()
eu_brand_model['Model'] = eu_brand_model['Model'].str.strip()


In [6]:
make_renaming_table = json.load(open("make_renaming.json"))
eu_brand_model['Make'] = eu_brand_model['Make'].map(lambda s: make_renaming_table.get(s) or s)

In [7]:
eu_brand_model.drop('Typ', axis=1, inplace=True)
eu_brand_model.replace('', np.nan, inplace=True)
eu_brand_model.dropna(inplace=True)

## Brand Specific Cleaning

In [8]:
eu_brand_model.groupby(['Make']).Cnt.sum().sort_values(ascending=False)

Make
VOLKSWAGEN           7902925
RENAULT              5235661
PEUGEOT              5174652
FORD                 4389648
MERCEDES-BENZ        3786680
                      ...   
REULT                    540
DFSK                     478
MPM MOTORS               240
MORGAN                   218
DACIA - ROMTURING        212
Name: Cnt, Length: 65, dtype: int64

In [9]:
top_models = eu_brand_model.loc[eu_brand_model.Cnt >= 5000].dropna()
top_models.loc[top_models['Make']=='AUDI'].Model.unique()

array(['A4 AVANT', 'A1 SPORTBACK', 'Q5', 'Q3', 'A6 AVANT', 'Q2',
       'A3 SPORTBACK', 'A5 SPORTBACK', 'Q8', 'A6 LIMOUSINE',
       'AUDI E-TRON', 'Q7', 'A1', 'A3 LIMOUSINE', 'Q3 SPORTBACK',
       'A4 LIMOUSINE', 'A4 ALLROAD QUATTRO', 'A1 SPORT NAV TFSI',
       'A6 ALLROAD QUATTRO', 'A6 S LINE 40 TDI S-A',
       'Q5 S LINE TDI QUATTRO S-A', 'SQ5', 'E-TRON 50', 'A4', 'A3', 'SQ7',
       'A3 CABRIOLET', 'RS 3 SPORTBACK', 'E-TRON 55', 'A1 SPORT 30 TFSI',
       'A7 SPORTBACK', 'A3 40 E-TRON', 'Q5 55 TFSI E',
       'A1 S LINE 30 TFSI', 'Q5 50 TFSI E', 'A1 S LINE NAV TFSI',
       'Q5 S LINE 40 TDI QUATTRO S-A', 'A5 CABRIOLET', 'S6 AVANT',
       'TT COUPE', 'A3 S LINE TFSI', 'RS3 SPORTBACK', 'A1 CITYCARVER',
       'A3 S LINE 35 TFSI S-A', 'E-TRON'], dtype=object)

In [ ]:
# AUDI
def audi(model: str):
    if model[0] in ['A', 'Q', 'S'] and model[:4] != "AUDI":
        return model.split(' ')[0]
    else:
        return model
    
top_models.loc[top_models['Make']=='AUDI'].Model.map(audi).unique()

array(['A4', 'A1', 'Q5', 'Q3', 'A6', 'Q2', 'A3', 'A5', 'Q8',
       'AUDI E-TRON', 'Q7', 'SQ5', 'E-TRON 50', 'SQ7', 'RS 3 SPORTBACK',
       'E-TRON 55', 'A7', 'S6', 'TT COUPE', 'RS3 SPORTBACK', 'E-TRON'],
      dtype=object)

In [ ]:
def bmw(model: str):
    if model[0].isdigit() or model[1].isdigit():
        return model.split(' ')[0]
    else:
        return model

top_models.loc[top_models['Make']=='BMW'].Model.map(bmw).unique()

array(['118I', 'X3', 'X1', 'I3', 'X5', '116D', '520D', '320D', '225XE',
       '118D', '530E', 'SERIE X', '218I', '318D', '116I', '320I', 'I3S',
       '330E', 'X2', 'X4', '120I', '330I', '420D', '530D', 'SERIE 1',
       '420I', 'M2', '216D', '218D', 'M140I', 'Z4', '220I', '318I',
       '540I', 'M4', '120D', '520I'], dtype=object)

In [11]:
eu_brand_model.sort_values(['Make', 'Make', 'Fuel', 'year']).to_csv("eu_emission_latest_with_year.csv")

In [12]:
df = pd.read_csv("eu_emission_latest_with_year.csv")

In [13]:
make_model_count = df.groupby(['Make', 'Model']).Cnt.sum().sort_values(ascending=False)
make_model_count

Make               Model                         
VOLKSWAGEN         GOLF                              1603729
RENAULT            CLIO                              1603607
PEUGEOT            208                               1243526
VOLKSWAGEN         POLO                              1052062
                   TIGUAN                            1050777
                                                      ...   
RENAULT            MEGANE / 1.5 / DCI                    212
DACIA - ROMTURING  LOGAN                                 212
VOLVO              XC40 R-DESIGN PRO T5 PHEV AUTO        212
MERCEDES-BENZ      B 180 EXCLUSIVE EDITION +             212
SKODA              KAROQ / 1.5 / TSI                     212
Name: Cnt, Length: 5560, dtype: int64

In [43]:
make_model_count[make_model_count>10000].reset_index().sort_values(by=["Make", "Model"]).to_csv("make_model.csv", index=False)

In [31]:
make_model_count = df.sort_values(by="Cnt", ascending=False)

In [32]:
make_model_count.loc[make_model_count['Cnt']>10000].sort_values(['Make', 'Model', 'Cnt']).to_csv("make_model_count.csv")